# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Summary" data-toc-modified-id="Summary-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Summary</a></div><div class="lev1 toc-item"><a href="#Version-Control" data-toc-modified-id="Version-Control-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Version Control</a></div><div class="lev1 toc-item"><a href="#Change-Log" data-toc-modified-id="Change-Log-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Change Log</a></div><div class="lev1 toc-item"><a href="#Setup" data-toc-modified-id="Setup-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Setup</a></div><div class="lev1 toc-item"><a href="#SymbolList()" data-toc-modified-id="SymbolList()-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>SymbolList()</a></div><div class="lev2 toc-item"><a href="#Web-service-call" data-toc-modified-id="Web-service-call-51"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Web service call</a></div><div class="lev3 toc-item"><a href="#Gather-elements" data-toc-modified-id="Gather-elements-511"><span class="toc-item-num">5.1.1&nbsp;&nbsp;</span>Gather elements</a></div><div class="lev3 toc-item"><a href="#Get-data" data-toc-modified-id="Get-data-512"><span class="toc-item-num">5.1.2&nbsp;&nbsp;</span>Get data</a></div><div class="lev3 toc-item"><a href="#Save-to-file" data-toc-modified-id="Save-to-file-513"><span class="toc-item-num">5.1.3&nbsp;&nbsp;</span>Save to file</a></div><div class="lev3 toc-item"><a href="#Data-inspection" data-toc-modified-id="Data-inspection-514"><span class="toc-item-num">5.1.4&nbsp;&nbsp;</span>Data inspection</a></div><div class="lev2 toc-item"><a href="#Helper-function" data-toc-modified-id="Helper-function-52"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Helper function</a></div><div class="lev3 toc-item"><a href="#Usage" data-toc-modified-id="Usage-521"><span class="toc-item-num">5.2.1&nbsp;&nbsp;</span>Usage</a></div><div class="lev2 toc-item"><a href="#Client-function" data-toc-modified-id="Client-function-53"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Client function</a></div>

# Summary

Part of the blog series related to making web service calls to Eoddata.com. Overview of the web service can be found [here](http://ws.eoddata.com/data.asmx).

 * ** View the master post of this series to build a secure credentials file.** It is used in all posts related to this series.
 * Download this blog post as a [jupyter notebook](https://adriantorrie.github.io/downloads/notebooks/eoddata/eoddata_web_service_calls_symbol_list.ipynb)
 * Download the [class definition file](https://adriantorrie.github.io/downloads/code/eoddata.py) for an easy to use client, which is demonstrated below
 * This post covers the `CountryList` call: http://ws.eoddata.com/data.asmx?op=SymbolList

# Version Control

In [1]:
%run ../../code/version_check.py

Python: 3.5.3 |Continuum Analytics, Inc.| (default, Feb 22 2017, 21:13:27) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]

matplotlib:	2.0.0
numpy:		1.12.0
pandas:		0.19.2
quandl:		3.0.1
sklearn:	0.18.1
scipy:		0.18.1
statsmodels:	0.8.0
tensorflow:	1.0.1


# Change Log

    Date Created: 2017-04-02
    
    Date of Change    Change Notes
    --------------    ----------------------------------------------------------------
    2017-04-02        Initial draft

# Setup

In [2]:
%run ../../code/eoddata.py

import pandas as pd
import requests as r

ws = 'http://ws.eoddata.com/data.asmx'
ns='http://ws.eoddata.com/Data'

with (Client()) as eoddata:
    token = eoddata.get_token()

In [3]:
exchange_code = 'NASDAQ'

# SymbolList()

## Web service call

In [4]:
session = r.Session()

call = 'SymbolList'
kwargs = {'Token': token, 'Exchange': exchange_code}
pattern = './/{%s}SYMBOL'

url = '/'.join((ws, call))
response = session.get(url, params=kwargs, stream=True)

if response.status_code == 200:
    root = etree.parse(response.raw).getroot()
    
session.close()

### Gather elements

In [5]:
elements = root.findall(pattern %(ns))

### Get data

In [6]:
symbols = sorted((element.get('Code'), element.get('Name')) for element in elements)
symbols[:10]

[('AAAP', 'Advanced Accele. Ads'),
 ('AAL', 'American Airlines Gp'),
 ('AAME', 'Atlantic Amer Cp'),
 ('AAOI', 'Applied Optoelect Cmn'),
 ('AAON', 'Aaon Inc'),
 ('AAPC', 'Atlantic Alliance Or'),
 ('AAPL', 'Apple Inc'),
 ('AAWW', 'Atlas Air Ww'),
 ('AAXJ', 'Ishares MSCI Aca X-Japan'),
 ('ABAC', 'Aoxin Tianli Grp')]

### Save to file

In [7]:
with open('../../data/nasdaq_symbols.csv', 'w') as f:
    for element in elements:
        f.write('"%s"\n' % '","'.join(element.attrib.values()))

### Data inspection

In [8]:
for item in root.items():
    print (item)

('Message', 'Success')
('Date', '0001-01-01T00:00:00')
('Source', 'Data.SymbolList')


In [9]:
for element in root.iter():
    print(element.attrib)

{'Message': 'Success', 'Date': '0001-01-01T00:00:00', 'Source': 'Data.SymbolList'}
{}
{'LongName': 'Advanced Accele. Ads', 'DateTime': '2017-03-31T00:00:00', 'Code': 'AAAP', 'Name': 'Advanced Accele. Ads'}
{'LongName': 'American Airlines Gp', 'DateTime': '2017-03-31T00:00:00', 'Code': 'AAL', 'Name': 'American Airlines Gp'}
{'LongName': 'Atlantic Amer Cp', 'DateTime': '2017-03-31T00:00:00', 'Code': 'AAME', 'Name': 'Atlantic Amer Cp'}
{'LongName': 'Applied Optoelect Cmn', 'DateTime': '2017-03-31T00:00:00', 'Code': 'AAOI', 'Name': 'Applied Optoelect Cmn'}
{'LongName': 'Aaon Inc', 'DateTime': '2017-03-31T00:00:00', 'Code': 'AAON', 'Name': 'Aaon Inc'}
{'LongName': 'Atlantic Alliance Or', 'DateTime': '2017-03-31T00:00:00', 'Code': 'AAPC', 'Name': 'Atlantic Alliance Or'}
{'LongName': 'Apple Inc', 'DateTime': '2017-03-31T00:00:00', 'Code': 'AAPL', 'Name': 'Apple Inc'}
{'LongName': 'Atlas Air Ww', 'DateTime': '2017-03-31T00:00:00', 'Code': 'AAWW', 'Name': 'Atlas Air Ww'}
{'LongName': 'Ishares M

## Helper function

In [10]:
def SymbolList(session, token, exchange_code):
    call = 'CountryList'
    kwargs = {'Token': token, 'Exchange': exchange_code}
    pattern = ".//{%s}CountryBase"

    url = '/'.join((ws, call))
    response = session.get(url, params=kwargs, stream=True)

    if response.status_code == 200:
        root = etree.parse(response.raw).getroot()
        
    return sorted((element.get('Code'), element.get('Name')) for element in elements)

### Usage

In [11]:
session = r.session()
symbols = SymbolList(session, token, exchange_code)
session.close()

symbols[:10]

[('AAAP', 'Advanced Accele. Ads'),
 ('AAL', 'American Airlines Gp'),
 ('AAME', 'Atlantic Amer Cp'),
 ('AAOI', 'Applied Optoelect Cmn'),
 ('AAON', 'Aaon Inc'),
 ('AAPC', 'Atlantic Alliance Or'),
 ('AAPL', 'Apple Inc'),
 ('AAWW', 'Atlas Air Ww'),
 ('AAXJ', 'Ishares MSCI Aca X-Japan'),
 ('ABAC', 'Aoxin Tianli Grp')]

## Client function

In [12]:
# pandas dataframe is returned
df = eoddata.symbol_list(exchange_code)

df.head()

,Code,Name,LongName,DateTime
0,AAAP,Advanced Accele. Ads,Advanced Accele. Ads,2017-03-31T00:00:00
1,AAL,American Airlines Gp,American Airlines Gp,2017-03-31T00:00:00
2,AAME,Atlantic Amer Cp,Atlantic Amer Cp,2017-03-31T00:00:00
3,AAOI,Applied Optoelect Cmn,Applied Optoelect Cmn,2017-03-31T00:00:00
4,AAON,Aaon Inc,Aaon Inc,2017-03-31T00:00:00
